In [1]:
#r "nuget: Microsoft.SemanticKernel, *-*"
#r "nuget: Microsoft.SemanticKernel.Planners.Handlebars, *-*"

Installed Packages Microsoft.SemanticKernel, 1.0.0-rc4 Microsoft.SemanticKernel.Planners.Handlebars, 1.0.0-rc4

In [2]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Memory;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.Planning;


In [3]:
#!import ../../Plugins/CustomPlugin/CompanySearchPlugin.cs

In [4]:
using System;
using System.Collections.Generic;
using System.Diagnostics;
using System.IO;
using System.Net.Http;
using System.Text.Json;
using System.Threading.Tasks;

In [5]:
Kernel kernel = Kernel.CreateBuilder()
            .AddAzureOpenAIChatCompletion("Your Azure OpenAI Service Deployment Name" , "Your Azure OpenAI Service Endpoint", "Your Azure OpenAI Service API Key")
            .Build();

In [6]:
var companySearchPluginObj = new CompanySearchPlugin();
var companySearchPlugin = kernel.ImportPluginFromObject(companySearchPluginObj, "CompanySearchPlugin");

In [7]:
var pluginDirectory = Path.Combine("../..", "plugins");

In [8]:
var writetPlugin =  kernel.ImportPluginFromPromptDirectory(Path.Combine(pluginDirectory, "WriterPlugin"));
var emailPlugin =  kernel.ImportPluginFromPromptDirectory(Path.Combine(pluginDirectory, "EmailPlugin"));
var translatePlugin =  kernel.ImportPluginFromPromptDirectory(Path.Combine(pluginDirectory, "TranslatePlugin"));

In [9]:
string goal =  """
Check the weather in Guangzhou and generate dressing tips based on the results,  and write emails
""";

In [10]:
using Microsoft.SemanticKernel.Planning.Handlebars;

#pragma warning disable SKEXP0060

var planner = new HandlebarsPlanner();

In [11]:
#pragma warning disable SKEXP0060

var originalPlan = await planner.CreatePlanAsync(kernel, goal);

In [12]:
Console.WriteLine(originalPlan);

{{!-- Step 1: Get weather information for Guangzhou --}}
{{set "weatherInfo" (CompanySearchPlugin-WeatherSearch text="Guangzhou")}}

{{!-- Step 2: Generate dressing tips based on the weather information --}}
{{set "dressingTips" (WriterPlugin-Tips input=(get "weatherInfo") language="en")}}

{{!-- Step 3: Write an email about the weather --}}
{{set "emailContent" (EmailPlugin-WeatherMail input=(get "weatherInfo"))}}

{{!-- Step 4: Print the final results --}}
{{json (get "emailContent")}}


In [13]:
originalPlan.ToString()

{{!-- Step 1: Get weather information for Guangzhou --}}
{{set "weatherInfo" (CompanySearchPlugin-WeatherSearch text="Guangzhou")}}

{{!-- Step 2: Generate dressing tips based on the weather information --}}
{{set "dressingTips" (WriterPlugin-Tips input=(get "weatherInfo") language="en")}}

{{!-- Step 3: Write an email about the weather --}}
{{set "emailContent" (EmailPlugin-WeatherMail input=(get "weatherInfo"))}}

{{!-- Step 4: Print the final results --}}
{{json (get "emailContent")}}

In [14]:
#pragma warning disable SKEXP0060
var originalPlanResult = await originalPlan.InvokeAsync(kernel, new KernelArguments());

In [15]:
Console.WriteLine(originalPlanResult);

Dear colleague,

I hope this email finds you well. I wanted to provide you with some information regarding the weather forecast for Guangzhou tomorrow.

According to the latest updates, tomorrow's temperature is expected to be around 2 degrees Celsius. Additionally, it's going to be a rainy day, so please make sure to bring an umbrella or raincoat with you when heading out.

Considering the low temperature, I would advise you to dress warmly by layering your clothing, such as wearing a sweater or jacket underneath your coat. Don't forget to wear a pair of comfortable shoes suitable for walking in wet conditions.

Furthermore, I would like to remind you to stay updated on the current travel situation. Please pay attention to any potential weather-related delays or issues that may impact your commute. It's always a good idea to check for any transportation updates before leaving and be prepared for possible delays.

In case there are any significant changes in the weather conditions or i